# Trade To Portfolio Rate (TTPR) Demo

*** DISCLAIMER ***

Please contact support@lusid.com if you want FINBOURNE to enable this functionality in your domain.

This notebook demonstrates how LUSID can resolve the Trade To Portfolio Rate for transactions booked with different currencies to the base portfolio currency.

In this example, 5 transactions in 5 different currencies are booked to a GBP portfolio and the associated TTPRs returned.

## 1. Setup LUSID

In [1]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import json
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Define the apis
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lusid.TransactionPortfoliosApi)

In [3]:
# Define variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC)
settlement_date = datetime(year=2022, month=3, day=12, tzinfo=pytz.UTC)
scope = "TTPR_NB"
portfolio_code = "TTPR_NB"

## 2. Create Portfolio

In [4]:
try: 
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name="TTPR_NB",
            code=portfolio_code,
            created="2020-01-01T00:00:00+00:00",
            base_currency="GBP"
        )
    )
    
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'TTPR_NB' because it already exists in scope 'TTPR_NB'.


## 3. Load Instruments

In [5]:
# Define 5 example instruments
for i in range(1, 6):
    instrument_body = models.InstrumentDefinition(
        name=f"instrument_{i}",
        identifiers={"ClientInternal": models.InstrumentIdValue(value=f"client_internal_{i}")}
    )
    
    reponse = instruments_api.upsert_instruments(
        request_body={
            "request_id_1": instrument_body
        }
    )

## 4. Load Quotes

### 4.1 FX Rate Quotes

In [6]:
# FX rate quote upsertion function
def spot_request(from_ccy, to_ccy, rate, valuation_date):
            return models.UpsertQuoteRequest(
                       quote_id=models.QuoteId(
                           models.QuoteSeriesId(
                               provider='Lusid',
                               instrument_id=f'{from_ccy}/{to_ccy}',
                               instrument_id_type='CurrencyPair',
                               quote_type='Rate',
                               field='mid'
                           ),
                           effective_at=valuation_date
                       ),
                       metric_value=models.MetricValue(
                           value=rate,
                           unit=f'{from_ccy}/{to_ccy}'
                       ),
                       lineage='None'
            )

In [7]:
fx_rates = [["USD", 1.3106], ["AUD", 1.7909], ["JPY", 151.14], ["EUR", 1.2073], ["CAD", 1.6803]]

for rate in fx_rates:
    response = quotes_api.upsert_quotes(scope=scope,
                                   request_body={"1": spot_request("GBP", rate[0], rate[1], valuation_date)})

## 5. Define Default Recipe

This is the default valuation recipe definition for the parameter added to the AWS parameter store for this domain enabling TTPR calculation.

In [8]:
# Create recipes
recipe_scope="IBOR"
recipe_code="StandardMarketValue"


# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope='Lusid',
            # This enables FX rate inference
            attempt_to_infer_missing_fx=True,

        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## 6. Load Transactions

In [9]:
# Upsert example transaction data for 5 instruments
cost = [15.25, 25.70, 57.34, 100.99, 23.19]
currency = ["USD", "AUD", "JPY", "EUR", "CAD"]

for i in range(1,6):
    transaction_portfolios_api.upsert_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_request=[models.TransactionRequest(
            transaction_id=f"transaction_id_{i}",
            type="Buy",
            instrument_identifiers={
                "instrument/default/ClientInternal" : f"client_internal_{i}"
            },
            transaction_date=valuation_date,
            settlement_date=settlement_date,
            units=100,
            total_consideration = models.CurrencyAndAmount(
                amount=cost[i-1],
                currency=currency[i-1],
            )
        )]
    )

## 7. Get Transactions

In [10]:
get_holdings_reponse = transaction_portfolios_api.get_holdings(
    scope = scope,
    code = portfolio_code,
    effective_at = valuation_date
)

get_transactions_reponse = transaction_portfolios_api.get_transactions(
    scope = scope,
    code = portfolio_code,
)

holdings_df_all = lusid_response_to_data_frame(get_holdings_reponse)
transactions_df_all = lusid_response_to_data_frame(get_transactions_reponse)
combined_df_all = transactions_df_all.merge(holdings_df_all, on='instrument_uid', how='left')

# Column selection
transactions_df = combined_df_all[[
    "transaction_id", 
    "type", 
    "instrument_identifiers.Instrument/default/ClientInternal",
    "instrument_uid",
    "transaction_currency",
    "cost_portfolio_ccy.currency",
    "properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value"
]]

# Column renaming
transactions_df = transactions_df.rename(
    columns={
        "transaction_id" : "transactionId", 
        "instrument_identifiers.Instrument/default/ClientInternal" : "ClientInternal",
        "instrument_uid" : "LUID",
        "transaction_currency" : "TradeCurrency",
        "cost_portfolio_ccy.currency" : "PortfolioCurrency",
        "properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value" : "TTPR"
    }
)

transactions_df

transactionId type     ClientInternal           LUID TradeCurrency  \
0  transaction_id_1  Buy  client_internal_1  LUID_00003EXA           USD   
1  transaction_id_2  Buy  client_internal_2  LUID_00003EXB           AUD   
2  transaction_id_3  Buy  client_internal_3  LUID_00003EXC           JPY   
3  transaction_id_4  Buy  client_internal_4  LUID_00003EXD           EUR   
4  transaction_id_5  Buy  client_internal_5  LUID_00003EXE           CAD   

  PortfolioCurrency      TTPR  
0               GBP  0.763009  
1               GBP  0.558378  
2               GBP  0.006616  
3               GBP  0.828295  
4               GBP  0.595132